# Semantic Web: SPARQL Language
_by Wael TELLAT_

## Ressources:
* Semantic Web course: http://www.eurecom.fr/~troncy/teaching/websem2017/ 
* To run queries: http://yasgui.org/ (or http://dbpedia.org/sparql)
* Wikidata queries: https://query.wikidata.org/
* A nice tutorial: http://www2.imm.dtu.dk/pubdb/views/edoc_download.php/6148/pdf

## Let's query the Web !
Some Questions we will try to answer to:
* What are the 20 biggest cities in the world in terms of their population ?
* What are the longitudes/latitudes of the 20 biggest french cities ?
* What are the countries with 'Republic' in their name that were established before 1920 ?
* How many people live in the capital of Australia ?
* Names of all lakes in Denmark ?
* How many Empoyees does PwC have ?
* Who wrote the French national anthem ?
* Did Tesla win the Nobel prize in physics ?


In [211]:
# The number of employees in PwC, using SPARQL

import requests, urllib

query = """
    PREFIX o: <http://dbpedia.org/ontology/>
    PREFIX r: <http://dbpedia.org/resource/>
    SELECT DISTINCT ?n
    WHERE
    {
        r:PricewaterhouseCoopers o:numberOfEmployees ?n
    }
"""
param = urllib.parse.urlencode({
        'format': 'text/csv', 
        'default-graph-uri': 'http://dbpedia.org', 
        'query': query
    }).encode("utf-8")
endpoint = 'http://dbpedia.org/sparql'

data = urllib.request.urlopen(endpoint, param).read().decode()
print(data)

"n"
208109



In [205]:
import json
import urllib
import requests
import csv

In [206]:
def send_query(query, format='text/csv'):
    # Query the DBpedia data in CSV format
    # Parameter format={'text/csv', 'json'}
    param = urllib.parse.urlencode({
            'format': format, 
            'default-graph-uri': 'http://dbpedia.org', 
            'query': query
        }).encode("utf-8")
    endpoint = 'http://dbpedia.org/sparql'
    data = urllib.request.urlopen(endpoint, param).read().decode()
    return data


In [207]:
def pretty_print(data, format='text/csv'):
    if format=='text/csv':
        for row in csv.reader(data.split('\n')):
            print(row)
    if format=='json':
        for r in json.loads(data_json)['results']['bindings']:
            print([r[k]['value'] for k in r.keys()])

In [208]:
# The 20 biggest cities in the world in terms of their population:
query = """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX o: <http://dbpedia.org/ontology/>
    PREFIX p: <http://dbpedia.org/property/>
    PREFIX r: <http://dbpedia.org/resource/>
    PREFIX f: <http://xmlns.com/foaf/0.1/>
    SELECT DISTINCT ?name ?n
    WHERE
    {
        ?x rdf:type o:City. 
        ?x p:populationTotal ?n .
        ?x p:name ?name
    }
    ORDER BY DESC(xsd:integer(?n))
    LIMIT 20
"""
data = send_query(query, format='text/csv')

In [209]:
pretty_print(data, format='text/csv')

['name', 'n']
['Imphal City', '264986414288']
['San Andrés Tuxtla', '61769157364']
['Rajauri', '41552642415']
['Mandla', '4946371579']
['Santiago Tuxtla', '1545956427']
['Karachi', '24300000']
['Delhi', '16314838']
['Mumbai', '12442373']
['Cairo', '10230350']
['Kinshasa', '10125000']
['Lahore', '10052000']
['Ameca', '10000000']
['New York City', '8491079']
['Bangalore', '8443675']
['Bogotá', '7878783']
['Chennai', '7088000']
['Dhaka', '6970105']
['Hyderabad', '6731790']
['Ahmedabad', '5577940']
['Muzaffarpur', '4801062']
[]


In [196]:
# The 20 biggest cities in the world in terms of their population:
query = """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX o: <http://dbpedia.org/ontology/>
    PREFIX p: <http://dbpedia.org/property/>
    PREFIX r: <http://dbpedia.org/resource/>
    PREFIX f: <http://xmlns.com/foaf/0.1/>
    PREFIX geo: <http://www.w3.org/2003/01/geo/wgs84_pos#>
    PREFIX yago: <http://dbpedia.org/class/yago/>
    
    SELECT DISTINCT ?label, ?n, ?lat, ?long
    WHERE
    {
        ?x rdf:type yago:City108524735. 
        ?x o:country r:France.
        ?x p:population ?n.
        ?x rdfs:label ?label.
        ?x geo:lat ?lat.
        ?x geo:long ?long.
        FILTER (lang(?label) = 'fr')
    }
    ORDER BY DESC(xsd:integer(?n))
    LIMIT 20
"""
data = send_query(query, format='text/csv')
print(data)

"label","n","lat","long"
"Paris",2240621,48.8567,2.3508
"Marseille",852516,43.2964,5.37
"Lyon",500715,45.76,4.84
"Toulouse",461190,43.6045,1.444
"Nice",343629,43.7034,7.2663
"Poitiers",298339,46.58,0.34
"Nantes",284970,47.2181,-1.5528
"Strasbourg",275718,48.58,7.75
"Montpellier",268456,43.6119,3.8772
"Bordeaux",241287,44.84,-0.58
"Lille",228652,50.6278,3.0583
"Rennes",211373,48.1147,-1.6794
"Reims",188078,49.2628,4.0347
"Saint-Étienne",178530,45.4347,4.3903
"Le Havre",175497,49.49,0.1
"Toulon",163974,43.13,5.92
"Grenoble",156659,45.2002,5.7222
"Dijon",152071,47.2906,5.04139
"Dijon",152071,47.2906,5.0428
"Dijon",152071,47.3219,5.04139



In [145]:
# The countries with 'Republic' in their name that were established before 1920
query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX o: <http://dbpedia.org/ontology/>
PREFIX p: <http://dbpedia.org/property/>
SELECT DISTINCT ?y ?d
WHERE
{
    ?x rdf:type o:Country.
    ?x o:longName ?y.
    ?x p:establishedDate ?d
    FILTER regex(?y, "Republic", "i" )
    MINUS {
        ?x p:establishedDate ?d
        FILTER(xsd:integer(?d)>1920)
    }
}
"""
data = send_query(query)
# pretty_print(data)

In [146]:
# The number of people who live in the capital of Australia
query = """
PREFIX o: <http://dbpedia.org/ontology/>
PREFIX p: <http://dbpedia.org/property/>
PREFIX r: <http://dbpedia.org/resource/>
SELECT DISTINCT ?n
WHERE
{
    r:Australia p:capital ?x.
    ?x o:populationTotal ?n
}
"""
data = send_query(query)
pretty_print(data)

['n']
['381488']
[]


In [151]:
# The name of all lakes in Denmark:
query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX o: <http://dbpedia.org/ontology/>
PREFIX p: <http://dbpedia.org/property/>
PREFIX r: <http://dbpedia.org/resource/>
SELECT DISTINCT ?label
WHERE
{
    { 
        ?x rdf:type o:Lake .
        ?x o:country r:Denmark .
        ?x rdfs:label ?label
        FILTER (lang(?label) = 'en')
    }
    UNION
    { ?x rdf:type r:LakesOfDenmark . }
}
"""
data = send_query(query)
pretty_print(data)

['l']
['Lake Bagsværd']
['The Lakes, Copenhagen']
['Bølling Lake']
['Furesø (lake)']
[]


In [152]:
# The number of employees in PwC
query = """
PREFIX o: <http://dbpedia.org/ontology/>
PREFIX r: <http://dbpedia.org/resource/>
SELECT DISTINCT ?n
WHERE
{
    r:PricewaterhouseCoopers o:numberOfEmployees ?n
}
"""
data = send_query(query)
pretty_print(data)

['n']
['208109']
[]


In [156]:
# The list of songs from Bruce Springsteen released between 1980 and 1990:
query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX o: <http://dbpedia.org/ontology/>
PREFIX p: <http://dbpedia.org/property/>
PREFIX r: <http://dbpedia.org/resource/>
SELECT DISTINCT *
WHERE
{
    {
      ?x o:artist r:Bruce_Springsteen.
      ?x p:name ?name
    }
    UNION {?x o:musicalArtist r:Bruce_Springsteen.}
    UNION {?x o:musicalBand r:Bruce_Springsteen.}
    UNION {?x o:producer r:Bruce_Springsteen.}
    
    ?x rdf:type o:Song .
    ?x o:releaseDate ?n .
    
    FILTER( YEAR(?n)<1990 && YEAR(?n)>1980 )
}
"""
data = send_query(query)
pretty_print(data)

['x', 'name', 'n']
['http://dbpedia.org/resource/No_Surrender_(song)', 'No Surrender', '1984-06-04']
['http://dbpedia.org/resource/Bobby_Jean', 'Bobby Jean', '1984-06-04']
['http://dbpedia.org/resource/Downbound_Train', 'Downbound Train', '1984-06-04']
['http://dbpedia.org/resource/Working_on_the_Highway', 'Working on the Highway', '1984-06-04']
['http://dbpedia.org/resource/Darlington_County_(song)', 'Darlington County', '1984-06-04']
['http://dbpedia.org/resource/Johnny_99_(song)', 'Johnny 99', '1982-09-30']
['http://dbpedia.org/resource/Downbound_Train', '', '1984-06-04']
['http://dbpedia.org/resource/Working_on_the_Highway', '', '1984-06-04']
['http://dbpedia.org/resource/Darlington_County_(song)', '', '1984-06-04']
['http://dbpedia.org/resource/Johnny_99_(song)', '', '1982-09-30']
[]


In [157]:
# The list of members of Prodigy (the band):
query = """
PREFIX o: <http://dbpedia.org/ontology/>
PREFIX r: <http://dbpedia.org/resource/>
SELECT DISTINCT *
WHERE
{
    r:The_Prodigy o:bandMember ?x.
}
"""
data = send_query(query)
pretty_print(data)

['x']
['http://dbpedia.org/resource/Liam_Howlett']
['http://dbpedia.org/resource/Keith_Flint']
['http://dbpedia.org/resource/Maxim_(musician)']
[]


In [162]:
# The author of the lyrics for the French national anthem:
query = """
PREFIX o: <http://dbpedia.org/ontology/>
PREFIX p: <http://dbpedia.org/property/>
PREFIX r: <http://dbpedia.org/resource/>
SELECT DISTINCT ?name
WHERE
{
    r:France o:anthem ?y.
    ?x o:occupation ?y.
    ?x p:name ?name
}
"""
data = send_query(query)
pretty_print(data)

['name']
['Claude Joseph Rouget de Lisle']
[]


In [165]:
# A query that tells if Tesla won the Nobel prize in physics:
query = """
PREFIX o: <http://dbpedia.org/ontology/>
PREFIX r: <http://dbpedia.org/resource/>
ASK
{
    r:Nikola_Tesla o:award r:Nobel_Prize_in_Physics.
}
"""
data = send_query(query)
pretty_print(data)

['bool']
['0']
[]


In [204]:
print("sth")

sth
